# How it works

**AnnCollectionDataset** directly reads h5ad files through [AnnCollection](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/anncollection.html) and serve data using [LitData](https://github.com/Lightning-AI/litdata) frontend. To use dataset, we first need to prepare dataset index folder that has multiple splits.

# Building LitData index

In [1]:
import os
from pathlib import Path

from bmfm_targets.datasets.anncollection.anncollection_dataset import (
    get_ann_collection,
)
from bmfm_targets.datasets.data_conversion.litdata_indexing import build_index

/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/pandera/engines/pandas_engine.py:67: UserWarning: Using typeguard < 3. Generic types like List[TYPE], Dict[TYPE, TYPE] will only validate the first element in the collection.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/dccstor/bmfm-targets/users/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def prepare_folders(h5ad_src, dest_root=None, dest=None):
    if dest is None and dest_root is not None:
        dest = Path(dest_root) / "_".join([l[1] for l in h5ad_src])
    else:
        dest = Path(dest)
    data_dir = dest / "ALL"
    os.makedirs(data_dir, exist_ok=True)
    for elm in h5ad_src:
        ln_src = Path(elm[0]) if type(elm) is tuple else Path(elm)
        name = f"{(Path(elm[1]) if type(elm) is tuple else Path(elm)).stem}.h5ad"
        ln_dst = data_dir / name
        if os.path.isfile(ln_dst):
            os.remove(ln_dst)
        os.symlink(ln_src, ln_dst)

    index_dir = dest / "litdata_index"
    os.makedirs(index_dir, exist_ok=True)
    #shutil.rmtree(index_dir)
    return {
        "index": index_dir,
        "data": data_dir,
    }

In [3]:
src_root = Path("/dccstor/bmfm-targets/data/omics/transcriptome/scRNA/finetune")
src_root

PosixPath('/dccstor/bmfm-targets/data/omics/transcriptome/scRNA/finetune')

In [4]:
dst_root = src_root / "AnnCollection"
dst_root

PosixPath('/dccstor/bmfm-targets/data/omics/transcriptome/scRNA/finetune/AnnCollection')

In [5]:
flds = prepare_folders([
    (src_root / "ChangEpi" / "processed.h5ad", "changepi"),
    (src_root / "SCP1884" / "processed.h5ad", "scp1884"),
    (src_root / "scIBD300K" / "processed.h5ad", "scp259"),
], dest_root=dst_root)
flds

{'index': PosixPath('/dccstor/bmfm-targets/data/omics/transcriptome/scRNA/finetune/AnnCollection/changepi_scp1884_scp259/litdata_index'),
 'data': PosixPath('/dccstor/bmfm-targets/data/omics/transcriptome/scRNA/finetune/AnnCollection/changepi_scp1884_scp259/ALL')}

In [6]:
dataset_dir = flds["data"]
index_dir = flds["index"]


### Reading hda5 files into annotation collection 
see https://anndata.readthedocs.io/en/latest/tutorials/notebooks/anncollection.html

In [7]:

collection = get_ann_collection(input_dir=dataset_dir, join_obs="outer")

/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [8]:
collection.adatas[0].obs

,Unnamed: 0,nUMI,nGene,log10GenesPerUMI,mitoRatio,cells,sample,patient,status,Full_Epi_Clus,TissueLoc,Final_CellType,UMAPfull_1,UMAPfull_2,S.Score,G2M.Score,Phase,split_stratified_Final_CellType,n_counts
HA01_AC_AAACCCAAGGCTGGAT.1,HA01_AC_AAACCCAAGGCTGGAT-1,2487,1016,0.885507,0.014073,HA01_AC_AAACCCAAGGCTGGAT-1,HA01_AC,HA01,NonIBD,7,Colon,TA_2,6.467434,-3.884951,0.494657,0.356172,S,dev,3601
HA01_AC_AAACCCAAGTCGGGAT.1,HA01_AC_AAACCCAAGTCGGGAT-1,17224,5053,0.874276,0.117627,HA01_AC_AAACCCAAGTCGGGAT-1,HA01_AC,HA01,NonIBD,7,Colon,TA_2,7.663868,-3.298563,-0.027053,0.561948,G2M,train,8112
HA01_AC_AAACCCAAGTGAACAT.1,HA01_AC_AAACCCAAGTGAACAT-1,23920,4825,0.841219,0.157609,HA01_AC_AAACCCAAGTGAACAT-1,HA01_AC,HA01,NonIBD,5,Colon,EC_3,-0.366386,6.462451,-0.040436,-0.077858,G1,train,6986
HA01_AC_AAACCCACAAGCGATG.1,HA01_AC_AAACCCACAAGCGATG-1,1499,743,0.904020,0.314209,HA01_AC_AAACCCACAAGCGATG-1,HA01_AC,HA01,NonIBD,6,Colon,GC_sub1,-2.295665,-10.757671,-0.065239,0.012134,G2M,train,2427
HA01_AC_AAACCCATCTACAGGT.1,HA01_AC_AAACCCATCTACAGGT-1,12209,3337,0.862156,0.147268,HA01_AC_AAACCCATCTACAGGT-1,HA01_AC,HA01,NonIBD,5,Colon,EC_3,-1.050888,5.104529,-0.049718,0.021243,G2M,train,5576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HA55_TI_TTTGTTGGTAGCCCTG.1,HA55_TI_TTTGTTGGTAGCCCTG-1,3003,1171,0.882389,0.373293,HA55_TI_TTTGTTGGTAGCCCTG-1,HA55_TI,HA55,NonIBD,1,Ileum,EC_5,-3.822792,-1.868396,-0.035682,-0.039043,G1,train,3734
HA55_TI_TTTGTTGGTCGTTATG.1,HA55_TI_TTTGTTGGTCGTTATG-1,4968,1726,0.875780,0.298309,HA55_TI_TTTGTTGGTCGTTATG-1,HA55_TI,HA55,NonIBD,1,Ileum,EC_5,-5.908050,-1.003698,-0.020253,-0.077762,G1,train,4284
HA55_TI_TTTGTTGGTGCCTATA.1,HA55_TI_TTTGTTGGTGCCTATA-1,4841,2058,0.899187,0.270812,HA55_TI_TTTGTTGGTGCCTATA-1,HA55_TI,HA55,NonIBD,1,Ileum,EC_5,-2.018079,1.109709,-0.057198,-0.045764,G1,train,4927
HA55_TI_TTTGTTGTCGGACAAG.1,HA55_TI_TTTGTTGTCGGACAAG-1,6660,2022,0.864601,0.383483,HA55_TI_TTTGTTGTCGGACAAG-1,HA55_TI,HA55,NonIBD,5,Ileum,EC_3,-0.254203,6.876445,-0.042127,-0.036100,G1,dev,4929


### Specify the column name for cell type classification for each adata

In [9]:
cell_type_columnname = [
    "Final_CellType",
    "Celltype",
    "celltype",
]

### Make index list for splits from processed h5ad

In [10]:
index_base = 0
train_index_all = []
dev_index_all = []
test_index_all = []
for i in range(len(collection.adatas)):
    print("length of",i,"th anndata=",len(collection.adatas[i]))
    adata = collection.adatas[i].obs.reset_index()
    train_index_ = list(adata[adata['split_stratified_'+cell_type_columnname[i]]=='train'].index)
    train_index = [x+index_base for x in train_index_]
    dev_index_ = list(adata[adata['split_stratified_'+cell_type_columnname[i]]=='dev'].index)
    dev_index = [x+index_base for x in dev_index_]
    test_index_ = list(adata[adata['split_stratified_'+cell_type_columnname[i]]=='test'].index)
    test_index = [x+index_base for x in test_index_]
    print("length of split (train, dev, test)", len(train_index), len(dev_index), len(test_index))
    
    index_base = index_base + len(adata)

    train_index_all = train_index_all + train_index
    dev_index_all = dev_index_all +dev_index
    test_index_all = test_index_all + test_index

length of 0 th anndata= 39023
length of split (train, dev, test) 31217 3903 3903
length of 1 th anndata= 720633


length of split (train, dev, test) 576505 72064 72064
length of 2 th anndata= 365492
length of split (train, dev, test) 292392 36550 36550


In [11]:
len(train_index_all), 31217+576505+292392

(900114, 900114)

In [12]:
len(dev_index_all), 3903+72064+36550

(112517, 112517)

### verification of index

In [13]:
ti_ = test_index_[-4:]
ti_

[365461, 365465, 365473, 365478]

In [14]:
ti=test_index[-4:]
ti

[1125117, 1125121, 1125129, 1125134]

In [ ]:
collection.adatas[1][ti_[3]].obs

,biosample_id,n_genes,n_counts,Chem,Site,Type,donor_id,Layer,Celltype,sex,...,organ,organ__ontology_label,disease,disease__ontology_label,batch,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,split_stratified_Celltype
index,,,,,,,,,,,,,,,,,,,,,
N10_LP_A-GACGAACTGGGACA,N10_LP_A,251,1130,v1,CO,Heal,N10,L,Plasma cells,unknown,...,UBERON_0001155,colon,PATO_0000461,normal,CO_IMM.scp,251,1052.0,43.0,4.087452,dev


In [16]:
collection.adatas[1][ti_[3]].obs['split_stratified_Celltype']

index
N10_LP_A-GACGAACTGGGACA    dev
Name: split_stratified_Celltype, dtype: category
Categories (1, object): ['dev']

In [17]:
collection.adatas[1][ti_[3]].obs.index

Index(['N10_LP_A-GACGAACTGGGACA'], dtype='object', name='index')

In [18]:
collection[ti[3]].obs['dataset'].index

Index(['N110.LPB.TGTCCCACATTAGCCA'], dtype='object')

In [19]:
"celltype" in collection.obs.columns

True

### Make folder for LitData index and create (test, dev) subfolders with LitData indices

Function build_index takes **index** parameter, an iterable such as Python generator

In [20]:
build_index(
    output_dir=index_dir / "train",
    index = train_index_all,
    chunk_size = 5000
)
build_index(
    output_dir=index_dir / "dev",
    index = dev_index_all,
    chunk_size = 5000
)
build_index(
    output_dir=index_dir / "test",
    index = test_index_all,
    chunk_size = 5000
)


# Testing dataset

In [21]:
from bmfm_targets import config
from bmfm_targets.datasets.anncollection import AnnCollectionDataModule
from bmfm_targets.tokenization import get_gene2vec_tokenizer

In [22]:
from importlib import reload

import bmfm_targets.datasets.anncollection

reload(bmfm_targets.datasets.anncollection)

<module 'bmfm_targets.datasets.anncollection' from '/dccstor/bmfm-targets/users/a1kosugi/gsetests/bmfm/bmfm_targets/datasets/anncollection/__init__.py'>

In [23]:
from importlib import reload

import bmfm_targets.datasets.anncollection.anncollection_dataset

reload(bmfm_targets.datasets.anncollection.anncollection_dataset)


<module 'bmfm_targets.datasets.anncollection.anncollection_dataset' from '/dccstor/bmfm-targets/users/a1kosugi/gsetests/bmfm/bmfm_targets/datasets/anncollection/anncollection_dataset.py'>

### Helper function that is needed only for tests

In [24]:
def gene2vec_fields():
    gene2vec_field_dicts = [
        {
            "field_name": "genes",
            "pretrained_embedding": None,
            "is_masked": False,
            "vocab_update_strategy": "static",
        },
        {
            "field_name": "expressions",
            "pretrained_embedding": None,
            "is_masked": True,
            "vocab_update_strategy": "static",
        },
    ]

    gene2vec_fields = [config.FieldInfo(**fd) for fd in gene2vec_field_dicts]
    tokenizer = get_gene2vec_tokenizer()
    for field in gene2vec_fields:
        field.update_vocab_size(tokenizer)
    return gene2vec_fields

### Parameters that normally have to be set in yaml file (see PanglaoDB yaml files).

In [25]:
dataset_kwargs = {
    "dataset_dir": dataset_dir,
    "index_dir": index_dir,
    "label_columns": ["Celltype", "celltype", "Final_CellType"],
}
tokenizer = get_gene2vec_tokenizer()
pars = {
    "tokenizer": tokenizer,
    "batch_size": 2,
    "fields": gene2vec_fields(),
    "num_workers": 8,
    "mlm": False,
    "collation_strategy": "sequence_classification",
    "dataset_kwargs": dataset_kwargs,
}

In [26]:
datamodule = AnnCollectionDataModule(**pars)
datamodule.prepare_data()
datamodule.setup("fit")


/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/u/a1kosugi/.conda/envs/bmfm_til/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [27]:
datamodule.label_dict

{'Celltype': {'Activated fibroblasts CCL19 ADAMADEC1': 0,
  'B cells': 1,
  'B cells AICDA LRMP': 2,
  'Cycling cells': 3,
  'DC1': 4,
  'DC2 CD1D': 5,
  'DC2 CD1D-': 6,
  'Endothelial cells CA4 CD36': 7,
  'Endothelial cells CD36': 8,
  'Endothelial cells DARC': 9,
  'Endothelial cells LTC4S SEMA3G': 10,
  'Enterochromaffin cells': 11,
  'Enterocytes BEST4': 12,
  'Enterocytes CA1 CA2 CA4-': 13,
  'Enterocytes TMIGD1 MEP1A': 14,
  'Enterocytes TMIGD1 MEP1A GSTA1': 15,
  'Enteroendocrine cells': 16,
  'Epithelial Cycling cells': 17,
  'Epithelial HBB HBA': 18,
  'Epithelial cells METTL12 MAFB': 19,
  'Fibroblasts ADAMDEC1': 20,
  'Fibroblasts KCNN3 LY6H': 21,
  'Fibroblasts NPY SLITRK6': 22,
  'Fibroblasts SFRP2 SLPI': 23,
  'Fibroblasts SMOC2 PTGIS': 24,
  'Glial cells': 25,
  'Goblet cells MUC2 TFF1': 26,
  'Goblet cells MUC2 TFF1-': 27,
  'Goblet cells SPINK4': 28,
  'IELs ID3 ENTPD1': 29,
  'ILCs': 30,
  'Immune Cycling cells': 31,
  'Inflammatory fibroblasts IL11 CHI3L1': 32,
  'L

## Look at a sample in dataset

In [28]:
dataset=datamodule.train_dataset

In [29]:
ds = dataset[1]
ds.metadata

{'Celltype': nan,
 'celltype': nan,
 'Final_CellType': 'EC_3',
 'cell_name': 'HA01_AC_AAACCCAAGTGAACAT.1'}

In [30]:
dataset[1000].metadata

{'Celltype': nan,
 'celltype': nan,
 'Final_CellType': 'SC',
 'cell_name': 'HA01_AC_CTTACCGCAATCGAAA.1'}

## Look at a collation result

In [32]:
train_dataloader = datamodule.train_dataloader()
itr = iter(train_dataloader)

In [33]:
for i in range(2):
    item = next(itr)
    c1 = dataset[i].metadata["cell_name"]
    c2 = item["cell_names"][0]
    print(f"{c1 == c2}: {c1} - {c2}")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


True: HA01_AC_AAACCCAAGTCGGGAT.1 - HA01_AC_AAACCCAAGTCGGGAT.1


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `_

False: HA01_AC_AAACCCAAGTGAACAT.1 - N128400_L-CAACCTCCAAACCACT


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [34]:
print(f"{c1 == c2}: {c1} - {c2}")

False: HA01_AC_AAACCCAAGTGAACAT.1 - N128400_L-CAACCTCCAAACCACT


In [35]:
train_dataloader = datamodule.train_dataloader()
item = next(iter(train_dataloader))
print(item)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `_

{'input_ids': tensor([[[    3,  8248,     0,  ...,  4761,  8316,     1],
         [    3,     7,     6,  ...,     6,     7,     1]],

        [[    3, 19247, 20318,  ..., 22585,  8066,     1],
         [    3,     6,     7,  ...,     6,     6,     1]]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'cell_names': ['HA01_AC_AAACCCAAGTCGGGAT.1', 'HA01_AC_AAACCCAAGTGAACAT.1'], 'labels': {'Celltype': tensor([-100, -100]), 'celltype': tensor([-100, -100]), 'Final_CellType': tensor([17,  6])}}
